In [ ]:
import pandas as pd
import numpy as np
from minisom import MiniSom
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10), place idx에서 23개가 비어있음
product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5834,11), product idx에서 538개가 비어있음
video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9), video idx에서 315개가 비어있음

In [ ]:
df_product = product.iloc[:,1:8].dropna()

In [ ]:
df_product = df_product.applymap(eval)  # Convert string representations of lists to actual lists

# Flatten the list columns for SOM input
def flatten_column(column):
    return pd.DataFrame(column.tolist(), index=column.index)

df_product_flat = pd.concat([flatten_column(df_product[col]) for col in df_product.columns], axis=1)

# Convert the dataframe to a numpy array
data = df_product_flat.values

In [ ]:
# SOM 초기화 및 학습
som = MiniSom(x=20, y=20, input_len=data.shape[1], sigma=1.0, learning_rate=0.5)
som.random_weights_init(data)
som.train_random(data, num_iteration=1000)

# 데이터 포인트가 매핑된 SOM의 위치 저장
mapped = np.array([som.winner(x) for x in data])
mapped_df = pd.DataFrame(mapped, columns=['x', 'y'])

# 군집 결과를 글로 출력
clusters = {}
for idx, loc in enumerate(mapped):
    loc = tuple(loc)
    if loc not in clusters:
        clusters[loc] = []
    clusters[loc].append(idx)

# 군집 결과 출력
for cluster, indices in clusters.items():
    print(f"Cluster {cluster}: {indices}")

# 결과 시각화
plt.figure(figsize=(14, 14))

# 배경으로 거리 맵 그리기
plt.pcolor(som.distance_map().T, cmap='coolwarm')
plt.colorbar(label='Distance Map')

# 데이터 포인트 그리기
colors = plt.cm.rainbow(np.linspace(0, 1, len(data)))
for i, x in enumerate(data):
    w = som.winner(x)  # 승자 노드 찾기
    plt.plot(w[0] + 0.5, w[1] + 0.5, 'o', markerfacecolor='None',
             markeredgecolor=colors[i], markersize=12, markeredgewidth=2)
    if i % 50 == 0:  # 숫자를 간헐적으로 표시하여 가독성 향상
        plt.text(w[0] + 0.5, w[1] + 0.5, str(i), color=colors[i],
                 fontdict={'weight': 'bold', 'size': 11}, ha='center', va='center')

plt.title('Self-Organizing Map (SOM) Visualization', fontsize=18)
plt.grid(True)
plt.show()

# 군집 결과를 데이터프레임으로 저장
cluster_df = pd.DataFrame.from_dict(clusters, orient='index').transpose()

In [ ]:
cluster_df